In [46]:
import os
import doctest
from llparser import *
from tokens import *
! python tokens.py

caching disabled: TestResults(failed=0, attempted=96)
caching  enabled: TestResults(failed=0, attempted=96)


In [2]:
os.chdir('../form/sources')
os.getcwd()

'/home/feelus/tmp/form/sources'

In [3]:
os.listdir()

['grccparam.h',
 'Makefile.am',
 'sort.c',
 'polygcd.h',
 'compiler.c',
 'names.c',
 'if.c',
 'transform.c',
 'evaluate.c',
 'sch.c',
 'comtool.c',
 'lus.c',
 'gentopo.cc',
 'poly.h',
 'proces.c',
 'token.c',
 'normal.c',
 'mallocprotect.h',
 'declare.h',
 'fsizes.h',
 'symmetr.c',
 'inivar.h',
 'parallel.c',
 'tools.c',
 'minos.h',
 'gentopo.h',
 'wildcard.c',
 'startup.c',
 'pattern.c',
 'minos.c',
 'pre.c',
 'function.c',
 'reshuf.c',
 'parallel.h',
 'polyfact.h',
 'grcc.cc',
 'message.c',
 'diagrams.c',
 'setfile.c',
 'float.c',
 'comtool.h',
 'diawrap.cc',
 'threads.c',
 'notation.c',
 'smart.c',
 'mpidbg.h',
 'grcc.h',
 'spectator.c',
 'ratio.c',
 'topowrap.cc',
 'mytime.cc',
 'polygcd.cc',
 'ftypes.h',
 'findpat.c',
 'mpi.c',
 'dollar.c',
 'opera.c',
 'bugtool.c',
 'unixfile.c',
 'tables.c',
 'unix.h',
 'vector.h',
 'polywrap.cc',
 'form3.h',
 'dict.c',
 'structs.h',
 'model.c',
 'extcmd.c',
 'module.c',
 'execute.c',
 'argument.c',
 'optimize.cc',
 'factor.c',
 'compcomm.c',
 '

In [4]:
file1 = open('form3.h', 'r')
Lines = file1.readlines()
Lines

['/** @file form3.h\n',
 ' *\n',
 ' *  Contains critical defines for the compilation process\n',
 ' *\tAlso contains the inclusion of all necessary header files.\n',
 ' *\tThere are also some system dependencies concerning file functions.\n',
 ' */\n',
 '\n',
 '/* #[ License : */\n',
 '/*\n',
 ' *   Copyright (C) 1984-2023 J.A.M. Vermaseren\n',
 ' *   When using this file you are requested to refer to the publication\n',
 ' *   J.A.M.Vermaseren "New features of FORM" math-ph/0010025\n',
 ' *   This is considered a matter of courtesy as the development was paid\n',
 ' *   for by FOM the Dutch physics granting agency and we would like to\n',
 ' *   be able to track its scientific use to convince FOM of its value\n',
 ' *   for the community.\n',
 ' *\n',
 ' *   This file is part of FORM.\n',
 ' *\n',
 ' *   FORM is free software: you can redistribute it and/or modify it under the\n',
 ' *   terms of the GNU General Public License as published by the Free Software\n',
 ' *   Foundation, e

In [5]:
Lines[0]

'/** @file form3.h\n'

In [6]:
def strip_baks_comment(lines):
    for l in lines:
        r = ''
        p=0
        while (q:=l[p:].find('/*$')) !=-1:
            r+=l[p:q]
            p = l[q+3].index('$*/')
        q = l[p:].find('//$') if '//$'in l else -1
        r+=l[p:q]
        yield r

In [7]:
def end_merge(lines):
    r=''
    sep_positions = []
    for l in lines:
        if not l.endswith('\\'):
            yield (r+l,sep_positions)
            r=''
            sep_positions=[]
        else:
            r+=l.removesuffix('\\')
            sep_positions.append(sep_positions[-1]+len(l)-1 if len(sep_positions) else len(l)-1)
    if len(sep_positions):
        raise ValueError('aaa')

In [8]:
for x,sp in end_merge(strip_baks_comment(Lines)):
    if len(sp):
        print(x)

#define STATIC_ASSERT__3(X,L) 	typedef char static_assertion_failed_##L[(!!(X))*2-1]
#define PADDUMMY(type, size) 	UBYTE d_u_m_m_y[alignof(type) - ((size) & (alignof(type) - 1))]
#define PADPOSITION(ptr_,long_,int_,word_,byte_) 	PADDUMMY(off_t, 		+ sizeof(int *) * (ptr_) 		+ sizeof(LONG)  * (long_) 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADPOINTER(long_,int_,word_,byte_) 	PADDUMMY(int *, 		+ sizeof(LONG)  * (long_) 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADLONG(int_,word_,byte_) 	PADDUMMY(LONG, 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADINT(word_,byte_) 	PADDUMMY(int, 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADWORD(byte_) 	PADDUMMY(WORD, 		+ sizeof(UBYTE) * (byte_) 	)
#define Utruncate(f) { 	if ( ftruncate(f->descriptor, 0) ) { 		MLOCK(ErrorMessageLock); 		MesPrint("Utruncate failed"); 		MUNLOCK(Er

In [9]:
def highlight_comments(lines):
    for l,sp in lines:
        pass

In [10]:
def printret(*args):
    print(*[repr(x) for x in args])
    return args[0]


In [11]:
x=7
try:
    x='123\u0100'.encode(encoding='latin1',errors='surrogatepass')
except Exception as e:
    print(type(e),*[(x,getattr(e,x)) for x in dir(e) if not x.startswith('__')],e,sep='\n')
x

<class 'UnicodeEncodeError'>
('add_note', <built-in method add_note of UnicodeEncodeError object at 0x7f8828194740>)
('args', ('latin-1', '123Ā', 3, 4, 'ordinal not in range(256)'))
('encoding', 'latin-1')
('end', 4)
('object', '123Ā')
('reason', 'ordinal not in range(256)')
('start', 3)
('with_traceback', <built-in method with_traceback of UnicodeEncodeError object at 0x7f8828194740>)
'latin-1' codec can't encode character '\u0100' in position 3: ordinal not in range(256)


7

In [12]:
def gen_fun(name):
    def default_name():
        return 1/0
    default_name.__name__ = name
    return default_name
foo = gen_fun('hello')
pass
pass
pass
foo()

ZeroDivisionError: division by zero